# 侦测走神司机
在此项目中，你将对 [数据集](https://www.kaggle.com/c/state-farm-distracted-driver-detection) 中的驾驶员状态图片进行分类。该数据集包含驾驶员十种状态下的图片。你需要预处理这些图片，然后用所有样本训练一个卷积神经网络。最后，你需要在样本图片上看到神经网络的预测结果。

## 获取图片

请运行以下单元，以下载 [数据集](https://www.kaggle.com/c/state-farm-distracted-driver-detection/)。将下载后的ZIP格式数据集解压到对应的目录

In [1]:
from urllib.request import urlretrieve 
from os.path import isfile, isdir 
from tqdm import tqdm 
import zipfile 
import csv
import os
import math  
import numpy as np  
import matplotlib.pyplot as plt     
from PIL import Image
from sklearn.model_selection import KFold
import pandas as pd
import shutil

cwd = os.getcwd()
print (cwd)

#图片数据路径
DRIVER_IMGS_PATH = "./distracted_driver_detection_dataset/"
#图片CSV列表的数据 
DRIVER_IMGS_LIST_CVS_DATE_FILE = "./distracted_driver_detection_dataset/driver_imgs_list.csv"
#图片的数据目录 
DRIVER_IMGES_DATASET_FOLDER_PATH = './distracted_driver_detection_dataset/imgs/'
#图片的ZIP位置 
DRIVER_IMGES_DATASET_LOCATION = './distracted_driver_detection_dataset/imgs.zip'
#图片训练集目录
TRAIN_DIR = './distracted_driver_detection_dataset/imgs/train/'
#图片测试集目录
TEST_DIR = './distracted_driver_detection_dataset/imgs/test/'
#训练好的模型位置
SAVE_MODEL_PATH = './saved_model/'

csv_file = csv.reader(open(DRIVER_IMGS_LIST_CVS_DATE_FILE,'r'))

if isfile(DRIVER_IMGES_DATASET_LOCATION): 
    zip_path = DRIVER_IMGES_DATASET_LOCATION 
else: 
    zip_path = 'imgs.zip'

        
if not isfile(zip_path): 
    with DLProgress(unit='B', unit_scale=True, miniters=1, desc='Dataset') as pbar:
        urlretrieve( 'https://www.kaggle.com/c/state-farm-distracted-driver-detection/data/imgs.zip', 
                    zip_path, 
                    pbar.hook)
        
#查看数据集imgs.zip是否已存在,解压到对应目录 
if not isdir(DRIVER_IMGES_DATASET_FOLDER_PATH): 
    with zipfile.ZipFile(zip_path) as zin: 
        zin.extractall(DRIVER_IMGES_DATASET_FOLDER_PATH) 
        zin.close()
        
print("all file is exits")

E:\pythonProject\machineLearningProgress\distracted_driver_detection
all file is exits


In [2]:
#读取CVS数据将图片分成训练集和验证集
def read_cvs_files(): 
    valid_subjects = ['p075','p081']
    if not os.path.exists(DRIVER_IMGS_PATH + "valid"):
        os.mkdir(DRIVER_IMGS_PATH + "valid")
        for i in range(10):
            os.mkdir(DRIVER_IMGS_PATH + "valid/c%d"%i)
    df = pd.read_csv(DRIVER_IMGS_LIST_CVS_DATE_FILE)
    for valid_subject in valid_subjects:
        df_valid = df[(df["subject"]==valid_subject)]
        for index, row in df_valid.iterrows():
            subpath = row["classname"] + "/" + row["img"]
            if os.path.exists(os.path.join(TRAIN_DIR,subpath)):
                shutil.move(os.path.join(TRAIN_DIR,subpath), os.path.join(DRIVER_IMGS_PATH,"valid/",subpath),)   
    print("valid dir is done")
read_cvs_files()

valid dir is done


## 加载数据集

In [7]:
import glob
import cv2
model_image_size = 224

print("-------- loading train data")
X_train = list()
y_train = list()
for i in range(10):
    dir = os.path.join(TRAIN_DIR, "c%d"%i)
    image_files = glob.glob(os.path.join(dir,"*.jpg"))
    print("loding {}, image count={}".format(dir, len(image_files)))
    for image_file in image_files:
        image = cv2.imread(image_file)
        X_train.append(cv2.resize(image, (model_image_size, model_image_size)))
        label = np.zeros(10, dtype=np.uint8)
        label[i]=1
        y_train.append(label)
X_train = np.array(X_train)
y_train = np.array(y_train)
        
print("-------- loading valid data")
X_valid = list()
y_valid = list()
for i in range(10):
    dir = os.path.join(DRIVER_IMGS_PATH, "valid/", "c%d"%i)
    image_files = glob.glob(os.path.join(dir,"*.jpg"))
    print("loding {}, image count={}".format(dir, len(image_files)))
    for image_file in image_files:
        image = cv2.imread(image_file)
        X_valid.append(cv2.resize(image, (model_image_size, model_image_size)))
        label = np.zeros(10, dtype=np.uint8)
        label[i]=1
        y_valid.append(label)
X_valid = np.array(X_valid)
y_valid = np.array(y_valid)

-------- loading train data
loding ./distracted_driver_detection_dataset/imgs/train/c0, image count=2308
loding ./distracted_driver_detection_dataset/imgs/train/c1, image count=2096
loding ./distracted_driver_detection_dataset/imgs/train/c2, image count=2136
loding ./distracted_driver_detection_dataset/imgs/train/c3, image count=2185
loding ./distracted_driver_detection_dataset/imgs/train/c4, image count=2160
loding ./distracted_driver_detection_dataset/imgs/train/c5, image count=2152
loding ./distracted_driver_detection_dataset/imgs/train/c6, image count=2164
loding ./distracted_driver_detection_dataset/imgs/train/c7, image count=1843
loding ./distracted_driver_detection_dataset/imgs/train/c8, image count=1771
loding ./distracted_driver_detection_dataset/imgs/train/c9, image count=1972
-------- loading valid data
loding ./distracted_driver_detection_dataset/valid\c0, image count=181
loding ./distracted_driver_detection_dataset/valid\c1, image count=171
loding ./distracted_driver_detec

In [8]:
print(X_train.shape)
print(y_train.shape)
print(X_valid.shape)
print(y_valid.shape)

(20787, 224, 224, 3)
(20787, 10)
(1637, 224, 224, 3)
(1637, 10)


In [2]:
from keras.models import *
from keras.optimizers import *
from keras.layers import *
from keras.applications import *
from keras.preprocessing.image import *

model_image_size = (224, 224)
fine_tune_layer = 18
final_layer = 21
visual_layer = 18
batch_size = 64

train_gen = ImageDataGenerator(
    samplewise_std_normalization=False,
    rotation_range=10.,
    width_shift_range=0.05,
    height_shift_range=0.05,
    shear_range=0.1,
    zoom_range=0.1,
)
gen = ImageDataGenerator(
    samplewise_std_normalization=False,
)

train_generator = train_gen.flow_from_directory(TRAIN_DIR,  model_image_size, shuffle=True, batch_size=batch_size, class_mode="categorical")
print("subdior to train type {}".format(train_generator.class_indices))
valid_generator = gen.flow_from_directory(os.path.join(DRIVER_IMGS_PATH, 'valid'),  model_image_size, shuffle=True, batch_size=batch_size, class_mode="categorical")
print("subdior to valid type {}".format(valid_generator.class_indices))

D:\ProgramData\Anaconda3\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


Found 20787 images belonging to 10 classes.
subdior to train type {'c0': 0, 'c1': 1, 'c2': 2, 'c3': 3, 'c4': 4, 'c5': 5, 'c6': 6, 'c7': 7, 'c8': 8, 'c9': 9}
Found 1637 images belonging to 10 classes.
subdior to valid type {'c0': 0, 'c1': 1, 'c2': 2, 'c3': 3, 'c4': 4, 'c5': 5, 'c6': 6, 'c7': 7, 'c8': 8, 'c9': 9}


## 构建模型

In [11]:
base_model = VGG16(input_tensor=Input((model_image_size, model_image_size, 3)), weights='imagenet', include_top=False)

for layers in base_model.layers:
    layers.trainable = False

x = GlobalAveragePooling2D()(base_model.output)
x = Dropout(0.25)(x)
x = Dense(10, activation='softmax')(x)
model = Model(base_model.input, x)
model.compile(optimizer='adadelta', loss='binary_crossentropy', metrics=['accuracy'])

#     model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
print("done")

done


In [ ]:
model.fit(X_train, y_train, batch_size=16, epochs=10, validation_data=(X_valid, y_valid))
model.save("models/vgg16-mymodel.h5")

Train on 20787 samples, validate on 1637 samples
Epoch 1/10


 3088/20787 [===>..........................] - ETA: 27:57:48 - loss: 1.0265 - acc: 0.843 - ETA: 23:49:21 - loss: 1.2245 - acc: 0.843 - ETA: 18:54:54 - loss: 1.1095 - acc: 0.847 - ETA: 15:51:45 - loss: 1.2225 - acc: 0.839 - ETA: 13:54:59 - loss: 1.2175 - acc: 0.831 - ETA: 12:45:21 - loss: 1.1987 - acc: 0.835 - ETA: 11:47:58 - loss: 1.2338 - acc: 0.831 - ETA: 11:00:40 - loss: 1.1879 - acc: 0.835 - ETA: 10:22:49 - loss: 1.1952 - acc: 0.833 - ETA: 9:53:28 - loss: 1.2029 - acc: 0.831 - ETA: 9:29:10 - loss: 1.2199 - acc: 0.83 - ETA: 9:08:10 - loss: 1.2237 - acc: 0.83 - ETA: 8:51:07 - loss: 1.2195 - acc: 0.83 - ETA: 8:36:38 - loss: 1.1965 - acc: 0.83 - ETA: 8:23:23 - loss: 1.1967 - acc: 0.83 - ETA: 8:13:47 - loss: 1.1812 - acc: 0.83 - ETA: 8:03:27 - loss: 1.1824 - acc: 0.83 - ETA: 7:55:11 - loss: 1.1695 - acc: 0.83 - ETA: 7:49:05 - loss: 1.1662 - acc: 0.83 - ETA: 7:44:41 - loss: 1.1755 - acc: 0.83 - ETA: 7:38:57 - loss: 1.1675 - acc: 0.83 - ETA: 7:34:19 - loss: 1.1626 - acc: 0.83 - ETA: 7:28:

 6192/20787 [=======>......................] - ETA: 4:54:00 - loss: 0.9890 - acc: 0.84 - ETA: 4:53:42 - loss: 0.9857 - acc: 0.84 - ETA: 4:53:24 - loss: 0.9821 - acc: 0.84 - ETA: 4:53:08 - loss: 0.9798 - acc: 0.84 - ETA: 4:52:48 - loss: 0.9783 - acc: 0.84 - ETA: 4:52:30 - loss: 0.9777 - acc: 0.84 - ETA: 4:52:10 - loss: 0.9771 - acc: 0.84 - ETA: 4:51:51 - loss: 0.9748 - acc: 0.84 - ETA: 4:51:33 - loss: 0.9736 - acc: 0.84 - ETA: 4:51:14 - loss: 0.9709 - acc: 0.84 - ETA: 4:50:56 - loss: 0.9707 - acc: 0.84 - ETA: 4:50:36 - loss: 0.9698 - acc: 0.84 - ETA: 4:50:18 - loss: 0.9702 - acc: 0.84 - ETA: 4:50:00 - loss: 0.9688 - acc: 0.84 - ETA: 4:49:44 - loss: 0.9685 - acc: 0.84 - ETA: 4:49:24 - loss: 0.9669 - acc: 0.84 - ETA: 4:49:05 - loss: 0.9652 - acc: 0.84 - ETA: 4:48:46 - loss: 0.9667 - acc: 0.84 - ETA: 4:48:29 - loss: 0.9656 - acc: 0.84 - ETA: 4:48:12 - loss: 0.9646 - acc: 0.84 - ETA: 4:47:54 - loss: 0.9633 - acc: 0.84 - ETA: 4:47:34 - loss: 0.9631 - acc: 0.84 - ETA: 4:47:16 - loss: 0.9633 -

 9296/20787 [============>.................] - ETA: 3:58:50 - loss: 0.8000 - acc: 0.85 - ETA: 3:58:34 - loss: 0.7989 - acc: 0.85 - ETA: 3:58:18 - loss: 0.7975 - acc: 0.85 - ETA: 3:58:01 - loss: 0.7966 - acc: 0.85 - ETA: 3:57:45 - loss: 0.7960 - acc: 0.85 - ETA: 3:57:28 - loss: 0.7951 - acc: 0.85 - ETA: 3:57:11 - loss: 0.7942 - acc: 0.85 - ETA: 3:56:55 - loss: 0.7937 - acc: 0.85 - ETA: 3:56:38 - loss: 0.7932 - acc: 0.85 - ETA: 3:56:21 - loss: 0.7925 - acc: 0.85 - ETA: 3:56:05 - loss: 0.7916 - acc: 0.85 - ETA: 3:55:49 - loss: 0.7906 - acc: 0.85 - ETA: 3:55:32 - loss: 0.7899 - acc: 0.85 - ETA: 3:55:15 - loss: 0.7890 - acc: 0.85 - ETA: 3:54:59 - loss: 0.7888 - acc: 0.85 - ETA: 3:54:42 - loss: 0.7883 - acc: 0.85 - ETA: 3:54:26 - loss: 0.7876 - acc: 0.85 - ETA: 3:54:10 - loss: 0.7870 - acc: 0.85 - ETA: 3:53:54 - loss: 0.7863 - acc: 0.85 - ETA: 3:53:38 - loss: 0.7856 - acc: 0.85 - ETA: 3:53:21 - loss: 0.7856 - acc: 0.85 - ETA: 3:53:05 - loss: 0.7848 - acc: 0.85 - ETA: 3:52:49 - loss: 0.7845 -

12400/20787 [================>.............] - ETA: 3:07:14 - loss: 0.6841 - acc: 0.86 - ETA: 3:06:58 - loss: 0.6837 - acc: 0.86 - ETA: 3:06:42 - loss: 0.6833 - acc: 0.86 - ETA: 3:06:26 - loss: 0.6825 - acc: 0.86 - ETA: 3:06:10 - loss: 0.6822 - acc: 0.86 - ETA: 3:05:55 - loss: 0.6818 - acc: 0.86 - ETA: 3:05:39 - loss: 0.6811 - acc: 0.86 - ETA: 3:05:23 - loss: 0.6810 - acc: 0.86 - ETA: 3:05:07 - loss: 0.6807 - acc: 0.86 - ETA: 3:04:51 - loss: 0.6802 - acc: 0.86 - ETA: 3:04:35 - loss: 0.6794 - acc: 0.86 - ETA: 3:04:20 - loss: 0.6793 - acc: 0.86 - ETA: 3:04:04 - loss: 0.6791 - acc: 0.86 - ETA: 3:03:48 - loss: 0.6786 - acc: 0.86 - ETA: 3:03:32 - loss: 0.6783 - acc: 0.86 - ETA: 3:03:17 - loss: 0.6777 - acc: 0.86 - ETA: 3:03:01 - loss: 0.6777 - acc: 0.86 - ETA: 3:02:45 - loss: 0.6771 - acc: 0.86 - ETA: 3:02:29 - loss: 0.6768 - acc: 0.86 - ETA: 3:02:13 - loss: 0.6762 - acc: 0.86 - ETA: 3:01:57 - loss: 0.6756 - acc: 0.86 - ETA: 3:01:41 - loss: 0.6758 - acc: 0.86 - ETA: 3:01:25 - loss: 0.6750 -

15504/20787 [=====================>........] - ETA: 2:16:33 - loss: 0.6003 - acc: 0.87 - ETA: 2:16:17 - loss: 0.5998 - acc: 0.87 - ETA: 2:16:01 - loss: 0.5994 - acc: 0.87 - ETA: 2:15:46 - loss: 0.5992 - acc: 0.87 - ETA: 2:15:30 - loss: 0.5989 - acc: 0.87 - ETA: 2:15:14 - loss: 0.5985 - acc: 0.87 - ETA: 2:15:00 - loss: 0.5979 - acc: 0.87 - ETA: 2:14:44 - loss: 0.5976 - acc: 0.87 - ETA: 2:14:28 - loss: 0.5973 - acc: 0.87 - ETA: 2:14:13 - loss: 0.5970 - acc: 0.87 - ETA: 2:13:57 - loss: 0.5963 - acc: 0.87 - ETA: 2:13:41 - loss: 0.5958 - acc: 0.87 - ETA: 2:13:25 - loss: 0.5954 - acc: 0.87 - ETA: 2:13:09 - loss: 0.5950 - acc: 0.87 - ETA: 2:12:54 - loss: 0.5947 - acc: 0.87 - ETA: 2:12:38 - loss: 0.5943 - acc: 0.87 - ETA: 2:12:22 - loss: 0.5938 - acc: 0.87 - ETA: 2:12:06 - loss: 0.5935 - acc: 0.87 - ETA: 2:11:51 - loss: 0.5931 - acc: 0.87 - ETA: 2:11:35 - loss: 0.5929 - acc: 0.87 - ETA: 2:11:19 - loss: 0.5925 - acc: 0.87 - ETA: 2:11:03 - loss: 0.5922 - acc: 0.87 - ETA: 2:10:48 - loss: 0.5918 -

18688/20787 [=========================>....] - ETA: 1:25:49 - loss: 0.5342 - acc: 0.88 - ETA: 1:25:34 - loss: 0.5340 - acc: 0.88 - ETA: 1:25:18 - loss: 0.5337 - acc: 0.88 - ETA: 1:25:02 - loss: 0.5336 - acc: 0.88 - ETA: 1:24:47 - loss: 0.5333 - acc: 0.88 - ETA: 1:24:31 - loss: 0.5329 - acc: 0.88 - ETA: 1:24:16 - loss: 0.5326 - acc: 0.88 - ETA: 1:24:00 - loss: 0.5323 - acc: 0.88 - ETA: 1:23:45 - loss: 0.5323 - acc: 0.88 - ETA: 1:23:29 - loss: 0.5320 - acc: 0.88 - ETA: 1:23:13 - loss: 0.5316 - acc: 0.88 - ETA: 1:22:58 - loss: 0.5314 - acc: 0.88 - ETA: 1:22:42 - loss: 0.5310 - acc: 0.88 - ETA: 1:22:27 - loss: 0.5308 - acc: 0.88 - ETA: 1:22:11 - loss: 0.5306 - acc: 0.88 - ETA: 1:21:55 - loss: 0.5303 - acc: 0.88 - ETA: 1:21:40 - loss: 0.5300 - acc: 0.88 - ETA: 1:21:24 - loss: 0.5298 - acc: 0.88 - ETA: 1:21:08 - loss: 0.5297 - acc: 0.88 - ETA: 1:20:53 - loss: 0.5294 - acc: 0.88 - ETA: 1:20:37 - loss: 0.5291 - acc: 0.88 - ETA: 1:20:21 - loss: 0.5288 - acc: 0.88 - ETA: 1:20:06 - loss: 0.5285 -

20784/20787 [============================>.] - ETA: 34:00 - loss: 0.4838 - acc: 0.89 - ETA: 33:44 - loss: 0.4836 - acc: 0.89 - ETA: 33:28 - loss: 0.4833 - acc: 0.89 - ETA: 33:13 - loss: 0.4831 - acc: 0.89 - ETA: 32:57 - loss: 0.4828 - acc: 0.89 - ETA: 32:41 - loss: 0.4827 - acc: 0.89 - ETA: 32:26 - loss: 0.4823 - acc: 0.89 - ETA: 32:10 - loss: 0.4821 - acc: 0.89 - ETA: 31:54 - loss: 0.4818 - acc: 0.89 - ETA: 31:39 - loss: 0.4817 - acc: 0.89 - ETA: 31:23 - loss: 0.4815 - acc: 0.89 - ETA: 31:07 - loss: 0.4814 - acc: 0.89 - ETA: 30:52 - loss: 0.4811 - acc: 0.89 - ETA: 30:36 - loss: 0.4809 - acc: 0.89 - ETA: 30:20 - loss: 0.4808 - acc: 0.89 - ETA: 30:05 - loss: 0.4805 - acc: 0.89 - ETA: 29:49 - loss: 0.4802 - acc: 0.89 - ETA: 29:33 - loss: 0.4801 - acc: 0.89 - ETA: 29:18 - loss: 0.4798 - acc: 0.89 - ETA: 29:02 - loss: 0.4795 - acc: 0.89 - ETA: 28:46 - loss: 0.4793 - acc: 0.89 - ETA: 28:31 - loss: 0.4790 - acc: 0.89 - ETA: 28:15 - loss: 0.4787 - acc: 0.89 - ETA: 27:59 - loss: 0.4784 - acc: 